First nights in hotels are always a bit of an anticlimax, what with the recovery from travel and all. You decided to do one of your wordsearch puzzles.

These puzzles are a bit different from normal because they have a puzzle grid and a list of words, but only some of the words are in the puzzle; some of the words given are decoys and aren't present.

For instance, given the grid:

```
fhjpamownq
wsdseuqeev
ieaeladhcr
piedtiriah
rzsiwovspu
oawhakieab
browpcfrda
ecnotopssr
dikchdnpnb
bstapleokr
```
and the list of words:

* adapting, apace, bombing, cackles, carnal, chump, coccyxes, cowhides, crazies, crumbled, dock, fickler, foaming, guts, knows, lived, minuend, molested, mown, pears, probed, rhubarb, rioted, shinier, solaria, staple, tops, wide, winced

you can find these words:

* apace, cowhides, crazies, dock, knows, lived, mown, pears, probed, rhubarb, rioted, staple, tops, wide

but these are the decoys:

* adapting, bombing, cackles, carnal, chump, coccyxes, crumbled, fickler, foaming, guts, minuend, molested, shinier, solaria, winced

For this puzzle, there are 14 words with a total length of 76 letters. (Some of the words may overlap in the grid, but don't worry about that when counting word lengths in your solution.)

## About wordsearches

Words can go in any of the eight directions (up, down, left, right, and diagonals) in a straight line. A letter in the grid can be in more than one word. Words don't wrap around the edges of the grid.

In the example above, the words "lived", "wide" and "staple" are in these positions (two words are diagonal and share a letter).

```
..........
.......e..
....l.d...
.....i....
....w.v...
.......e..
........d.
..........
..........
.staple...
```

The longest word, "cowhides", runs vertically upwards:

```
..........
...s......
...e......
...d......
...i......
...h......
...w......
...o......
...c......
..........
```

If there are words present in the grid that aren't in the list of words given, ignore them. For instance, you can see the word "brow" running left to right on the seventh row of the grid, but that doesn't count as a word in this puzzle because "brow" isn't in the list of words you're given.

You're safe to assume that each word in the puzzle is present either zero or one times, never more.

## File format
The wordsearch puzzle is given as a text file. The first line of the file is WxH, where W and H are the width and height of the puzzle grid, in characters. The next H lines are the grid, each line being W characters long. Finally, there's an arbitrary number of words to look for, one on each line.

Ignore any trailing or leading blank lines, and any whitespace on a line.

The example puzzle above, a ten by ten grid, would be written to a file as:

```
10x10
fhjpamownq
wsdseuqeev
ieaeladhcr
piedtiriah
rzsiwovspu
oawhakieab
browpcfrda
ecnotopssr
dikchdnpnb
bstapleokr
adapting
apace
bombing
cackles
carnal
chump
coccyxes
cowhides
crazies
crumbled
dock
fickler
foaming
guts
knows
lived
minuend
molested
mown
pears
probed
rhubarb
rioted
shinier
solaria
staple
tops
wide
winced
```

# Part 1

Your wordsearch puzzle is given in [10-wordsearch.txt](10-wordsearch.txt). 

What is the total of the lengths of all the words present in the puzzle?

After you've solved the wordsearch and found all the words you can, you'll have some letters unused in any word. For the example wordsearch, once you've found the words, you're left with this:

```
fhj.a....q
w....uq..v
i.a....h..
..e....i..
..........
....a.....
....p.f...
........s.
.i..h.npn.
b......okr
```
The letters remaining in the grid are `aaabeffhhhiiijknnoppqqrsuvw`. 9 of those letters are vowels. 

# Part 2

Your wordsearch puzzle is still given in [10-wordsearch.txt](10-wordsearch.txt).

How many vowels are left over after solving this puzzle?

# Worked solution: Part 1
After all the algorithmic headscratching of the previous couple of days, time for something a bit more sedate. 

## Data structures
This is fairly obvious. The wordsearch grid is just a grid of letters. I could store it as
* a list of lists of letters
* a list of strings
* a dict of letters, with key being the (row, column) pair/2-tuple

As I'll be updating the grid in part 2, it makes sense to have something mutable, so the list of strings will be awkward. Beyond that, there's not much to choose, so I went for a list of lists of letters. 

I use the `enum` library to build a new type for directions. This makes the program a bit clearer, as well as prevents typos if I have to refer to a particular direction. The `delta` dict converts from a direction to the row and column changes that move one step in that direction. 

## Examining the grid
With a standard string or list, Python (along with many other languages) has the idea of a _slice_: a section of the list identified by its start position and length. The function `gslice` (for "grid slice") does the same thing: return a slice of grid, defined by starting row and column, length, and direction. All `gslice` does is return the cell called out by `indices`. `indices` uses the `delta` dict to convert the direction into the row and column changes, and also does the bounds checking to only return (row, column) pairs that are in the grid.

`present_many` just does an exhaustive search of the grid, finding all possible grid slices (all rows, column, directions, and valid lengths) and checking if the slice returned is one of the given words. 

(There's another version of `present_many` that uses comprehensions to find the words, but that's slower as it keeps having to call `gslice` and `indices` for checking.)

# Worked solution: Part 2
This builds on part 1. First, I find all the words. I then take all those word positions and, on a copy of the grid, replace the characters in the word position with dots, using the `set_grid` procedure. Then it's just a matter of concatenating the grid into one long string, filtering to keep only the vowels, and counting how many there are.

In [1]:
import string
import re
import collections
import copy
import os

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [2]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [3]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [4]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [5]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [6]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [24]:
def present_many(grid, words):
    w = len(grid[0])
    h = len(grid)
    wordlens = set(len(w) for w in words)
    presences = []
    for r in range(h):
        for c in range(w):
            for d in Direction:
                for wordlen in wordlens:
                    word = cat(gslice(grid, r, c, wordlen, d))
                    if len(word) == wordlen and word in words:
                        presences += [(word, r, c, d)]
    return set(presences)

In [25]:
def present_many_c(grid, words):
    w = len(grid[0])
    h = len(grid)
    wordlens = set(len(w) for w in words)
    presences = set((cat(gslice(grid, r, c, wordlen, d)), r, c, d)
                    for r in range(h)
                    for c in range(w)
                    for d in Direction
                    for wordlen in wordlens
                    if len(indices(grid, r, c, wordlen, d)) == wordlen
                    if cat(gslice(grid, r, c, wordlen, d)) in words )
    return set(presences)

In [10]:
def read_wordsearch(fn):
    lines = [l.strip() for l in open(fn).readlines()]
    w, h = [int(s) for s in lines[0].split('x')]
    grid = lines[1:h+1]
    words = set(lines[h+1:])
    return w, h, grid, words

# All wordsearch puzzles

In [11]:
def read_all_wordsearch(fn):
    with open(fn) as f:
        text = f.read().strip()
        puzzles_text = text.split('\n\n')
        puzzles = []
        for p in puzzles_text:
            lines = p.splitlines()
            w, h = [int(s) for s in lines[0].split('x')]
            grid = lines[1:h+1]
            words = lines[h+1:]
            puzzles += [(w, h, grid, words)]
    return puzzles

## Huge wordsearch

In [12]:
puzzle = read_wordsearch('10-wordsearch.txt')
puzzle[:2]

(100, 100)

## Part 1

In [26]:
def found_words_length(puzzle):
    width, height, grid, words = puzzle
    return sum(len(p[0]) for p in present_many(grid, words))

def total_found_words_length(puzzles):
    return sum(found_words_length(p) for p in puzzles)

In [27]:
def found_words_length_c(puzzle):
    width, height, grid, words = puzzle
    return sum(len(p[0]) for p in present_many_c(grid, words))

def total_found_words_length_c(puzzles):
    return sum(found_words_length_c(p) for p in puzzles)

In [28]:
found_words_length(puzzle)

8092

In [29]:
%%timeit
found_words_length(puzzle)

1 loop, best of 3: 7.4 s per loop


In [30]:
found_words_length_c(puzzle)

8092

In [31]:
%%timeit
found_words_length_c(puzzle)

1 loop, best of 3: 12 s per loop


In [32]:
width, height, grid, words = puzzle
presences = present_many(grid, words)
found_words = [p[0] for p in presences]
len(presences), len(found_words), len(set(found_words))

(1149, 1149, 1149)

In [33]:
%%timeit
present_many(grid, words)

1 loop, best of 3: 7.4 s per loop


In [34]:
%%timeit
present_many_c(grid, words)

1 loop, best of 3: 11.9 s per loop


In [35]:
found_words = [p[0] for p in presences]
len(found_words), len(set(found_words))

(1149, 1149)

## Part 2

In [15]:
def max_unfound_word_length(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
    used_words = [p[0] for p in presences]
    unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_letters = [c for l in unused_grid for c in l if c != '.']
    unused_letter_count = collections.Counter(unused_letters)
    
    makeable_words = []
    for w in unused_words:
        unused_word_count = collections.Counter(w)
        if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
            makeable_words += [w]
    lwm = max(len(w) for w in makeable_words)
    return lwm

In [16]:
def unused_letters(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
#     used_words = [p[0] for p in presences]
#     unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_letters = [c for l in unused_grid for c in l if c != '.']
    unused_letter_count = collections.Counter(unused_letters)
    
    return used_words, unused_letter_count

In [17]:
def unused_vowels(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
#     used_words = [p[0] for p in presences]
#     unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_vowel_count = sum(1 for l in unused_grid for c in l if c in 'aeiou')
    return unused_vowel_count

In [18]:
def total_max_unfound_word_length(puzzles):
    return sum(max_unfound_word_length(p) for p in puzzles)

In [19]:
unused_vowels(puzzle)

594

In [20]:
%%timeit
unused_vowels(puzzle)

1 loop, best of 3: 30.8 s per loop


In [21]:
# max_unfound_word_length(puzzle)

In [22]:
# %%timeit
# max_unfound_word_length(puzzle)

In [23]:
uw, unlc = unused_letters(puzzle)
sum(unlc[l] for l in unlc)

2217